Задача  - разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. 

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. 

### Содержание

##### Часть 1. Загрузка необходимых библиотек и данных:
* [1. Загрузка данных.](#1)

##### Часть 2. Умножение матриц:
* [1. Математическое обоснование умножения на матрицу.](#2)
* [2. Обратное преобразование данных после умножения на матрицу.](#3)

##### Часть 3. Алгоритм преобразования:
* [1. Теория.](#4)

##### Часть 4. Проверка алгоритм преобразования:
* [1. Разделение на признаки.](#5)
* [2. Умножение на матрицу.](#6)
* [3. Запуск Линейной регрессии и сравнение данных.](#7)
* [4. Обратное преобразование.](#8)


## 1. Загрузка данных

In [1]:
# Подгрузка библиотек
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error



<a id='1'></a>

In [2]:
# Посмотрим на данные
data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv', sep=',')

for data, name in zip([data], ['data']):
    print(f'Размер датафрейма {name} - {data.shape}')
    for column in (data.columns):
        data[column].value_counts()
        print("Пропусков по колонке:", column," - ", data[column]\
                                    .isna().sum(), ", доля: {:.2%}"\
                                    .format(data[column].isna().sum()/len(data[column]))) 
    display(data.info())
    print("------------------------------------------- ")


Размер датафрейма data - (5000, 5)
Пропусков по колонке: Пол  -  0 , доля: 0.00%
Пропусков по колонке: Возраст  -  0 , доля: 0.00%
Пропусков по колонке: Зарплата  -  0 , доля: 0.00%
Пропусков по колонке: Члены семьи  -  0 , доля: 0.00%
Пропусков по колонке: Страховые выплаты  -  0 , доля: 0.00%
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
Пол                  5000 non-null int64
Возраст              5000 non-null float64
Зарплата             5000 non-null float64
Члены семьи          5000 non-null int64
Страховые выплаты    5000 non-null int64
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


None

------------------------------------------- 


пропусков нет, данные имеют правильный тип - это хороший признак)

Посмотрим на данные

In [3]:
# посмотрим на все данные в таблице и на целевой признак
display(data.head(10))
print('Целевой признак имеет следующие вариации: \n', data['Страховые выплаты'].value_counts())

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0
5,1,43.0,41000.0,2,1
6,1,39.0,39700.0,2,0
7,1,25.0,38600.0,4,0
8,1,36.0,49700.0,1,0
9,1,32.0,51700.0,1,0


Целевой признак имеет следующие вариации: 
 0    4436
1     423
2     115
3      18
4       7
5       1
Name: Страховые выплаты, dtype: int64


## 2. Умножение матриц <a id='2'></a>

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

### поехали ------->>>>>>>>>



1. Формула обучения дана в таком виде:
$$
w = (X^T X)^{-1} X^T y
$$


2. Представим $X_1$ как произведение старой X на матрицу M:
$$
X_1 = XM
$$


3. Подставим новое значение X в формулу $w$:
$$
w1 = ((XM)^T XM)^{-1} (XM)^T y
$$

4. Раскроем первое произведение $ (XM)^T $:
$$
w1 = (M^T X^T X M)^{-1} M^T X^T y
$$

5. Раскроем скобки :
$$
w1 = ((X^TХM)^{-1}(M^T)^{-1})M^T X^Ty
$$
$$
w1 = (M)^{-1}(X^TХ)^{-1}(M^T)^{-1}M^T X^Ty
$$

6. Поскольку матрица M обратима, то произведение $(M^T)^{-1} M^T$ равно E:  
$$
w1 = M^{-1}(X^TХ)^{-1}X^Ty 
$$

7. Заметим, что образовалась первоначальная формула $ w $: 
$$
w1 = M^{-1}w 
$$

8. Новый вектор предсказаний будет иметь вид:
$$
a1=X_1w1=XMM^{-1}w=Xw
$$

Судя по данной выкладки мы должны получить точно такие же предсказания и метрики, после умножения на обратимую матрицу(случайную), как и до умножения.


Таргет «Страховые выплаты» в данном случае принимает значения 1,2,3,4,5. Если эти значения упорядочены, то есть значения 5 больше значения 4, то порой имеет смысл использовать регрессионную модель, так как она будет учитывать, что 5 больше 4, а модель классификации - нет. Проверим это встроенной в библиотеку sklearn Линейной Регрессией.

Создадим функции для предсказания и предсказания

In [4]:
# функция моделей и вывода результата
def model_predict(model_type, features, target, features_valid, target_valid, name):
    """
    функция построения и проверки модели: 
    model_type - тип модели
    features -  признаки 
    target -  целевой признак
    features_valid -  валидационный признак
    target_valid - валидационный целевой признак
    name - имя для отображения на принте
    На выход функция выдаст: оценку по созданной метрике оценки, модель для оценки.
    
    """
    # модель 
    model = model_type
    # Обучим модель для остального добра
    model.fit(features,target)
    predict = model.predict(features_valid)
    print('Значения ', name)
    # метрики
    r2_value = r2_score(target_valid, predict)
    mse_value = mean_squared_error(target_valid, predict)
    print("R2 метрика =  : {:.8f} ".format(r2_value))
    print("MSE метрика =  : {:.8f} ".format(mse_value))
    return(predict, mse_value, r2_value)
    
    
# Функция для разделения на признаки 
def great_separator(df, column, test_size, name):
    """
    функция для назначения признаков по: 
    column -  целевая колонка признака
    df -  датафрейм для работы
    test_size -  размер тестовой области
    name -  имя целевого признака для отображения в принте
    на выход функция выдаст 4 объекта :
    1-features_train
    2-features_valid
    3-target_train
    4-target_valid
    
    """
    target = df[column]
    features = df.drop([column], axis=1)
    features_train, features_valid, target_train, target_valid = train_test_split(features,\
                                                                             target, test_size=test_size,random_state=12345)
    print('Размер обучающей области {} : {:.1%}'.format(name, len(features_train) / len(df)))
    print('Размер валидационной области {} : {:.1%}'.format(name, len(features_valid) / len(df)))
    return(features_train, features_valid, target_train, target_valid)    

<a id='4'></a>

преобразуем наши данные в исходный вид и увидим, связь параметров
формула 
$$
X_1 = XP
$$
$$
X = X_1P^{-1}
$$


**Ответ:** Видно, что качество Линейной регрессии не изменилось после операции умножения признаков Х на случайную матрицу.

**Обоснование:** Параметры линейной регрессии в исходной и в преобразованной связаны правилами умножения матриц. При матричном умножении  по двум матрицам строится третья. Она состоит из скалярных произведений строк первой матрицы на столбцы второй. По сути мы получили преобразованные по "определенному ключу" данные которые сохранили свои свойства для предсказания целевого признака. но при этому стали полностью анонимны. После обратного преобразования мы получили те же данные.

## 3. Алгоритм преобразования
<a id='4'></a>

**Алгоритм**
Тут мне видится только ранее использванный способ умножения на матрицу:
* 1) создаем и сохраняем матрицу - она будет ключом для дальнейшего обратного преобразования
* 2) проверяем матрицу на обратимость - она обязательно должна быть обратимой
* 3) умножаем наши данные на матрицу
* 4) запускаем машину для предсказаний 
* 5) преобразовываем наши данные обратно по ключу, если нужно получить доступ к данным для работы
...

**Обоснование**
* При такой операции преобразования данные сохранятся в виде векторов с значениями недоступными для восприятия с точки зрения пользователя и получения какой то личной информации. Но при этом они сохранят свойства векторов начальных данных необходимых для работы машинного обучения и предсказания нужной нам величины.
...

## 4. Проверка алгоритма
<a id='5'></a>

Я не придумал пока - нового алгоритма для преобразования. Я смог проверить и доказать только умножение на матрицу и верность метрик в 2 пункте этой работы. Возможно если есть какие то более конкретные идеи, то я думаю я смогу их воплотить в код. ) Буду рад замечаниям.

Запускаем ранее опробованный алгоритм

In [5]:
# разделим наши выборки на таргет и признаки для 'Страховые выплаты'
features_train, features_test, target_train, target_test = great_separator(data, 'Страховые выплаты', 0.25, 'Страховка')

Размер обучающей области Страховка : 75.0%
Размер валидационной области Страховка : 25.0%


In [6]:
# Для воспроизводимости результатов зафиксируем RandomState:
state = np.random.RandomState(12345)
# создадим случайную матрицу и проверим ее на обратимость
M = state.normal(2, 3, size=(4, 4))
# проверка на обратимость
try:
    np.linalg.inv(M)
except:
    print('Матрица необратимая')
print(f'Случайная матрица имеет вид \n {M}')

Случайная матрица имеет вид 
 [[ 1.38587702  3.43683001  0.44168385  0.33280909]
 [ 7.89734172  6.1802175   2.27872363  2.84523846]
 [ 4.3070677   5.73930421  5.02156807 -1.88866333]
 [ 2.8249749   2.68673864  6.05875051  4.65928802]]


In [7]:
# Запустим линейную регрессию
predict_before,mse_before, r2_before = model_predict(LinearRegression(),\
                                        features_train, target_train, features_test, target_test, 'Начальные данные ')

Значения  Начальные данные 
R2 метрика =  : 0.43522757 
MSE метрика =  : 0.11660517 


<a id='6'></a>

In [8]:
# умножим наши данные на матрицу 
features_train_M = np.dot(features_train,M)
features_test_M = np.dot(features_test,M)

оценим наши данные на анонимность )

In [9]:
# отобразим
display(features_train_M)

array([[ 156259.64738842,  208034.68529265,  181885.24978397,
         -68242.27509877],
       [ 248356.99517874,  330797.48667253,  289320.23927763,
        -108689.93673792],
       [ 177276.0224964 ,  236085.85669503,  206465.42569666,
         -77528.35583485],
       ...,
       [ 192853.92817909,  256806.41063551,  224564.02095508,
         -84301.60385805],
       [ 215969.13332945,  287685.85261838,  251654.92736661,
         -94540.8003014 ],
       [ 175878.41176832,  234281.03586616,  204923.27311713,
         -77003.40422463]])

<a id='7'></a>

In [10]:
# Запустим линейную регрессию на умноженных данных
model_predict(LinearRegression(), features_train_M, target_train,\
                                                                features_test_M, target_test, 'после умножения признаков')
print('----------------------------------')
# Оценка значений до умножения
print("Значения до умножения")
print("R2 метрика = ", round(r2_before, 8))
print("MSE метрика = ", round(mse_before,8))

Значения  после умножения признаков
R2 метрика =  : 0.43522757 
MSE метрика =  : 0.11660517 
----------------------------------
Значения до умножения
R2 метрика =  0.43522757
MSE метрика =  0.11660517


преобразуем наши данные в исходный вид и увидим, связь параметров
формула 
$$
X_1 = XP
$$
$$
X = X_1P^{-1}
$$
<a id='8'></a>

In [11]:
# создадим имена колонок
new_columns = ['Пол', 'Возраст', 'Зарплата', 'Члены семьи']
# выполним обратное преобразование по формуле
df_1 = np.dot(features_train_M ,np.linalg.inv(M))
df_2 = np.dot(features_test_M ,np.linalg.inv(M))
# округлим
df_1 = np.round(df_1).astype(int)
df_2 = np.round(df_2).astype(int)
# Соберем в датафрейм
df_1 = pd.DataFrame(data=df_1, columns=new_columns)
df_2 = pd.DataFrame(data=df_2, columns=new_columns)
# объеденим в 1
df  = pd.concat([df_1, df_2]).reset_index(drop=True)
# оценим
display('Данные до преобразований: ', data[data['Возраст']==40.0].sort_values(by='Зарплата', ascending=False).head(5))
print('')
display('Данные после преобразований: ', df[df['Возраст']==40].sort_values(by='Зарплата', ascending=False).head(5))


'Данные до преобразований: '

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
3732,0,40.0,63800.0,0,0
943,0,40.0,62100.0,1,0
71,0,40.0,61800.0,1,0
3271,1,40.0,58200.0,0,0
4112,0,40.0,56300.0,1,0


'Данные после преобразований: '

,Пол,Возраст,Зарплата,Члены семьи
1123,0,40,63800,0
4573,0,40,62100,1
664,0,40,61800,1
259,1,40,58200,0
1259,0,40,56300,1


**Ответ:** Видно, что качество Линейной регрессии не изменилось после операции умножения признаков Х на случайную матрицу.

**Обоснование:** Параметры линейной регрессии в исходной и в преобразованной связаны правилами умножения матриц. При матричном умножении  по двум матрицам строится третья. Она состоит из скалярных произведений строк первой матрицы на столбцы второй. По сути мы получили преобразованные по "определенному ключу" данные которые сохранили свои свойства для предсказания целевого признака. но при этому стали полностью анонимны. После обратного преобразования мы получили те же данные.

Алгоритм работает, можно запускать в работу)

